In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer

In [101]:
gene_ex = pd.read_csv("SingaporeCRC_data/SG-BULK_salmonTPM.csv", index_col=0, delimiter=",") # Load raw data
gene_ex = gene_ex[gene_ex.sum(axis = 1) > 0] # remove columns that all values are 0
gene_ex.head

<bound method NDFrame.head of                       1504       1603       1324        904        834  \
patient_id                                                               
ENSG00000000003  21.899490  23.051914  17.759606  19.887544  10.718835   
ENSG00000000005   0.201798   0.336181   0.321224   0.495977   0.092844   
ENSG00000000419  24.813475  32.488898  73.592613  17.660135  22.659393   
ENSG00000000457   2.385601   3.491298   3.189865   3.426733   2.741735   
ENSG00000000460   2.423479   4.838602   4.491900   2.546456   3.737818   
...                    ...        ...        ...        ...        ...   
ENSG00000273487   0.827590   0.683941   0.834527   0.857493   0.663991   
ENSG00000273488   1.265823   2.057645   1.717848   1.950747   2.913155   
ENSG00000273489   0.232299   0.171759   0.297231   0.535864   0.153239   
ENSG00000273492   1.413037   1.414762   1.526064   1.535159   1.518512   
ENSG00000273493   0.000000   0.000000   0.000000   0.609845   0.206916   

       

In [102]:
# Merge ensemble ids to gene names
gene_names = pd.read_csv("SingaporeCRC_data/gene_names.csv", index_col= 0,  delimiter=";")
gene_names

,patient_id,external_gene_name
1,ENSG00000000003,TSPAN6
2,ENSG00000000005,TNMD
3,ENSG00000000419,DPM1
4,ENSG00000000457,SCYL3
5,ENSG00000000460,FIRRM
...,...,...
52675,ENSG00000273415,LINC02725
52679,ENSG00000273423,OR13I1P
52683,ENSG00000273434,OR8S21P
52706,ENSG00000273484,OR6R2P


In [115]:
# Replace ensemble_id with corresponding gene names
gene_ex_new = pd.merge(gene_ex, gene_names, on='patient_id', how='inner')
gene_ex_new = gene_ex_new.drop(['patient_id'], axis = 1)
cols = gene_ex_new.columns.tolist()
cols = [cols[-1]] + cols[:-1]
gene_ex_new = gene_ex_new[cols]
gene_ex_new
#gene_ex_new.to_csv('gene_expre.csv', index=False)

,external_gene_name,1504,1603,1324,904,834,47,1477,1298,1037,...,1646,1076,1066,1464,1165,1607,1448,891,858,1614
0,TSPAN6,21.899490,23.051914,17.759606,19.887544,10.718835,10.178540,15.664737,6.391909,8.080226,...,12.060430,19.493468,8.679017,18.199760,6.008192,23.451535,22.468734,5.567352,12.708013,17.797957
1,TNMD,0.201798,0.336181,0.321224,0.495977,0.092844,0.162886,0.979404,0.240270,0.091914,...,0.301725,1.416982,0.039388,0.160707,0.085678,0.108787,0.633003,0.110154,0.473068,0.157542
2,DPM1,24.813475,32.488898,73.592613,17.660135,22.659393,9.907228,26.220157,8.971716,8.427556,...,14.959888,38.668035,19.646647,19.240425,16.253209,17.911861,23.361645,10.611309,9.516005,11.413409
3,SCYL3,2.385601,3.491298,3.189865,3.426733,2.741735,3.501638,3.163172,3.273375,2.039018,...,7.841108,4.182547,4.466471,5.350908,4.772413,3.298083,4.886279,7.648380,3.676741,5.313049
4,FIRRM,2.423479,4.838602,4.491900,2.546456,3.737818,1.934962,5.378430,2.950694,2.152042,...,4.288870,6.068358,3.514042,4.984912,4.211540,4.157520,5.240244,4.337681,2.140315,1.901577
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39640,LINC02725,0.000000,0.009547,0.000000,0.000000,0.010522,0.018542,0.013174,0.000000,0.000000,...,0.013752,0.119002,0.011689,0.015649,0.166215,0.032562,0.000000,0.018027,0.009558,0.013621
39641,OR13I1P,0.425036,0.414414,0.426886,0.467496,0.384386,0.569888,0.335258,0.267601,0.242140,...,0.490705,0.457142,0.424344,0.521561,0.407356,0.435850,0.698116,0.444446,0.489217,0.631813
39642,OR8S21P,0.000000,0.000000,0.000000,0.196300,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.186462,0.000000,0.142449,0.000000,0.161062,0.000000,0.459419,0.222001,0.000000,0.000000
39643,OR6R2P,2.029988,1.558693,1.920728,1.759279,1.185637,1.398731,2.261764,1.013812,0.838034,...,1.561123,1.401636,1.588408,3.166684,2.340912,2.041876,1.793439,2.078749,1.389800,2.492183


In [190]:
gene_expre = pd.read_csv("gene_expre.csv", index_col=0, delimiter=",")

# Impute missing values
imputer = KNNImputer(n_neighbors=5, weights="uniform")
gene_expre_imputed = imputer.fit_transform(gene_expre.T).T
gene_ex_imputed = pd.DataFrame(gene_expre_imputed, columns=gene_expre.columns)
gene_ex_final = gene_ex_imputed.T
gene_ex_final.columns = gene_expre.T.columns
gene_ex_final.shape

(162, 39645)

In [191]:
# Add a column containing original patient id for later "merge" step
pid = gene_ex_final.index.astype('float')
gene_ex_final.insert(0,'patient_id1',pid)
gene_ex_final

external_gene_name,patient_id1,TSPAN6,TNMD,DPM1,SCYL3,FIRRM,FGR,CFH,FUCA2,GCLC,...,OR2Q1P,TMEM167B-DT,LINC01396,OR5B15P,LINC02712,LINC02725,OR13I1P,OR8S21P,OR6R2P,APP-DT
1504,1504.0,21.899490,0.201798,24.813475,2.385601,2.423479,1.719561,4.938426,17.340649,17.525062,...,0.074462,0.331290,0.087628,0.000000,1.055453,0.000000,0.425036,0.000000,2.029988,1.413037
1603,1603.0,23.051914,0.336181,32.488898,3.491298,4.838602,1.709233,5.097042,23.091196,25.009775,...,0.000000,0.398889,0.020872,0.000000,1.003464,0.009547,0.414414,0.000000,1.558693,1.414762
1324,1324.0,17.759606,0.321224,73.592613,3.189865,4.491900,0.835416,2.342624,26.887325,30.539995,...,0.013473,0.365116,0.000000,0.000000,0.867961,0.000000,0.426886,0.000000,1.920728,1.526064
904,904.0,19.887544,0.495977,17.660135,3.426733,2.546456,0.686355,1.877872,16.863628,19.586386,...,0.065556,0.559125,0.132928,0.000000,1.652483,0.000000,0.467496,0.196300,1.759279,1.535159
834,834.0,10.718835,0.092844,22.659393,2.741735,3.737818,0.692069,1.440818,10.085834,14.951673,...,0.022015,0.445526,0.135381,0.000000,0.798360,0.010522,0.384386,0.000000,1.185637,1.518512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1607,1607.0,23.451535,0.108787,17.911861,3.298083,4.157520,0.483072,1.646369,19.915323,25.855965,...,0.000000,0.296483,0.035570,0.000000,1.374271,0.032562,0.435850,0.000000,2.041876,1.164592
1448,1448.0,22.468734,0.633003,23.361645,4.886279,5.240244,0.450576,1.690369,15.562312,19.087965,...,0.017427,0.261538,0.074034,0.000000,1.202577,0.000000,0.698116,0.459419,1.793439,1.737887
891,891.0,5.567352,0.110154,10.611309,7.648380,4.337681,5.249050,6.239189,15.364255,18.859650,...,0.000000,0.352240,0.076502,0.009575,1.175638,0.018027,0.444446,0.222001,2.078749,1.425941
858,858.0,12.708013,0.473068,9.516005,3.676741,2.140315,0.687859,2.102874,14.072503,14.159075,...,0.019996,0.537431,0.000000,0.000000,1.414841,0.009558,0.489217,0.000000,1.389800,1.933930


In [192]:
metadata = pd.read_csv('SingaporeCRC_data/SG-BULK_patient_clinical_information.csv', index_col=0,delimiter=",")
metadata = metadata.dropna()

In [193]:
pid_meta = metadata.index.astype('float')
metadata.insert(0,'patient_id1',pid_meta)
metadata

,patient_id1,TMB,KRAS,BRAF,NRAS,TP53,APC,PIK3CA,PIK3R1,SMAD4,...,Age.at.Diagnosis,Site.of.Primary.Colorectal.tumour,Side,Grade,TNM,Stage,iCMS,CMS,group3,group5
patient_id,,,,,,,,,,,,,,,,,,,,,
106,106.0,1028,wt,wt,wt,wt,wt,wt,wt,wt,...,72.0,Transverse colon,Right,3,T4aN2(4/29)M0,IIIC,iCMS3,CMS1,iCMS3_MSI,iCMS3_MSI
153,153.0,80,mut,wt,wt,mut,mut,mut,wt,wt,...,66.0,Sigmoid colon,Left,2,T3N1(1/18)M0,IIIB,iCMS2,CMS4,iCMS2_MSS,iCMS2_fibrotic
269,269.0,10,wt,wt,wt,wt,wt,wt,wt,wt,...,54.0,Rectum,Left,2,T1N1c(0/11)M0,IIIB,iCMS3,CMS4,iCMS3_MSS,iCMS3_fibrotic
326,326.0,1512,mut,wt,wt,wt,wt,wt,wt,wt,...,55.0,Rectosigmoid junction,Left,2,Unknown,II,iCMS3,CMS1,iCMS3_MSI,iCMS3_MSI
420,420.0,1976,mut,wt,wt,mut,mut,wt,wt,wt,...,58.0,Sigmoid colon,Left,2,T3N1(1/23)M0,IIIB,iCMS3,CMS1,iCMS3_MSI,iCMS3_MSI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1658,1658.0,82,mut,wt,wt,mut,mut,wt,wt,wt,...,59.0,Rectum,Left,2,T3N2b(8/22)M1,IV,iCMS2,CMS2,iCMS2_MSS,iCMS2_MSS
1662,1662.0,74,mut,wt,wt,mut,wt,wt,wt,wt,...,67.0,Rectum,Left,2,T3N1a(1/36)M0,IIIB,iCMS3,CMS3,iCMS3_MSS,iCMS3_MSS
1664,1664.0,46,mut,wt,wt,mut,wt,wt,wt,wt,...,62.0,Sigmoid colon,Left,2,T3N2a(6/29)M0,IIIC,iCMS2,CMS4,iCMS2_MSS,iCMS2_fibrotic


In [201]:
import re
#regex = re.compile(r"\[|\]|<", re.IGNORECASE)

SG_CRC_gene = pd.merge(gene_ex_final, metadata, on='patient_id1', how='inner')
#SG_CRC_gene.columns = [regex.sub("_",col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in SG_CRC_gene.columns.values]
SG_CRC_gene = pd.DataFrame(SG_CRC_gene)
SG_CRC_gene
#SG_CRC_gene.to_csv('merged_CRC_gene', index = False)


,patient_id1,TSPAN6,TNMD,DPM1,SCYL3,FIRRM,FGR,CFH,FUCA2,GCLC,...,Age.at.Diagnosis,Site.of.Primary.Colorectal.tumour,Side,Grade,TNM,Stage,iCMS,CMS,group3,group5
0,1504.0,21.899490,0.201798,24.813475,2.385601,2.423479,1.719561,4.938426,17.340649,17.525062,...,70.0,Rectum,Left,2,T2N1(3/21)M1,IV,iCMS2,CMS4,iCMS2_MSS,iCMS2_fibrotic
1,1603.0,23.051914,0.336181,32.488898,3.491298,4.838602,1.709233,5.097042,23.091196,25.009775,...,66.0,Rectosigmoid junction,Left,2,T3N0(0/19)M0,IIA,iCMS2,CMS2,iCMS2_MSS,iCMS2_MSS
2,1324.0,17.759606,0.321224,73.592613,3.189865,4.491900,0.835416,2.342624,26.887325,30.539995,...,73.0,Sigmoid colon,Left,2,T3N1c(0/17)M0,IIIB,iCMS2,CMS2,iCMS2_MSS,iCMS2_MSS
3,834.0,10.718835,0.092844,22.659393,2.741735,3.737818,0.692069,1.440818,10.085834,14.951673,...,71.0,Rectosigmoid junction,Left,2,T3N0(0/27)M0,IIA,iCMS2,CMS2,iCMS2_MSS,iCMS2_MSS
4,1662.0,27.208802,0.745929,14.048898,5.442878,1.954963,2.571285,7.385909,15.042006,14.375950,...,67.0,Rectum,Left,2,T3N1a(1/36)M0,IIIB,iCMS3,CMS3,iCMS3_MSS,iCMS3_MSS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,1607.0,23.451535,0.108787,17.911861,3.298083,4.157520,0.483072,1.646369,19.915323,25.855965,...,67.0,Ascending colon,Right,2,T3N1b(2/32)M0,IIIB,iCMS3,CMS3,iCMS3_MSS,iCMS3_MSS
109,1448.0,22.468734,0.633003,23.361645,4.886279,5.240244,0.450576,1.690369,15.562312,19.087965,...,68.0,Rectum,Left,2,T3N2(4/17)M0,IIIB,iCMS2,CMS2,iCMS2_MSS,iCMS2_MSS
110,891.0,5.567352,0.110154,10.611309,7.648380,4.337681,5.249050,6.239189,15.364255,18.859650,...,73.0,Splenic flexure,Left,2,T3N1(1/19)M0,IIA,iCMS3,CMS4,iCMS3_MSS,iCMS3_fibrotic
111,858.0,12.708013,0.473068,9.516005,3.676741,2.140315,0.687859,2.102874,14.072503,14.159075,...,55.0,Ascending colon,Right,2,T3N0(0/26)M0,IIA,iCMS3,CMS3,iCMS3_MSI,iCMS3_MSI


In [214]:
# Export final pre-processed human gene expression data
SG_CRC_gene.to_csv('SingaporeCRC_data/SG_CRC_GE.csv', index=False)